<a href="https://colab.research.google.com/github/kondou-shouichi-hro/kondou-shouichi-hro.github.io/blob/master/genome_hail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hailのインストール

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!apt-get install g++ liblz4-dev libopenblas liblapack3
!pip install hail

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libopenblas
     |████████████████████████████████| 93.1MB 38kB/s 
     |████████████████████████████████| 9.5MB 22.6MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 7.7MB 32.1MB/s 
     |████████████████████████████████| 133kB 47.3MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 212.4MB 64kB/s 
     |████████████████████████████████| 32.4MB 72kB/s 
     |████████████████████████████

## 実験データのダウンロード

In [2]:
%%shell
wget http://www.iu.a.u-tokyo.ac.jp/lectures/AG16/180509/RnData.zip
unzip RnData.zip
gunzip ./RnData/HDRA-G6-4-RDP1-RDP2-NIAS.AGCT_MAF005MIS001NR.vcf.gz

--2021-06-29 01:12:23--  http://www.iu.a.u-tokyo.ac.jp/lectures/AG16/180509/RnData.zip
Resolving www.iu.a.u-tokyo.ac.jp (www.iu.a.u-tokyo.ac.jp)... 49.212.198.234
Connecting to www.iu.a.u-tokyo.ac.jp (www.iu.a.u-tokyo.ac.jp)|49.212.198.234|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10405475 (9.9M) [application/zip]
Saving to: ‘RnData.zip’

RnData.zip          100%[===================>]   9.92M  7.95MB/s    in 1.2s    

2021-06-29 01:12:25 (7.95 MB/s) - ‘RnData.zip’ saved [10405475/10405475]

Archive:  RnData.zip
   creating: RnData/
  inflating: RnData/RiceDiversityPheno4GWASGS.csv  
   creating: __MACOSX/
   creating: __MACOSX/RnData/
  inflating: __MACOSX/RnData/._RiceDiversityPheno4GWASGS.csv  
  inflating: RnData/HDRA-G6-4-RDP1-RDP2-NIAS.AGCT_MAF005MIS001NR.vcf.gz  
  inflating: RnData/.DS_Store        
  inflating: __MACOSX/RnData/._.DS_Store  
  inflating: RnData/performGWAS.R    
  inflating: __MACOSX/RnData/._performGWAS.R  
  inflating: RnData/Ric

## Hailの初期化

In [3]:
%matplotlib inline
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Running on Apache Spark version 3.1.2
SparkUI available at http://abd78ed38748:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.70-5bb98953a4a7
LOGGING: writing to /content/hail-20210629-0112-0.2.70-5bb98953a4a7.log


## 遺伝子型データと表現型データの読み込み

In [4]:
mt = hl.import_vcf('./RnData/HDRA-G6-4-RDP1-RDP2-NIAS.AGCT_MAF005MIS001NR.vcf')
table_line = hl.import_table('./RnData/RiceDiversityLine4GWASGS.csv', delimiter=",",  key="", impute=True)
table_pheno = hl.import_table('./RnData/RiceDiversityPheno4GWASGS.csv', delimiter=",",  key="", impute=True)

2021-06-29 01:12:47 Hail: INFO: Reading table to impute column types
2021-06-29 01:12:52 Hail: INFO: Finished type imputation
  Loading field '' as type str (imputed)
  Loading field 'NSFTV.ID' as type int32 (imputed)
  Loading field 'GSOR.ID' as type int32 (imputed)
  Loading field 'IRGC.ID' as type str (imputed)
  Loading field 'Accession.Name' as type str (imputed)
  Loading field 'Country.of.origin' as type str (imputed)
  Loading field 'Latitude' as type float64 (imputed)
  Loading field 'Longitude' as type float64 (imputed)
  Loading field 'Sub.population' as type str (imputed)
2021-06-29 01:12:52 Hail: INFO: Reading table to impute column types
2021-06-29 01:12:53 Hail: INFO: Loading 37 fields. Counts by type:
  float64: 29
  int32: 7
  str: 1


In [5]:
# 遺伝子型と表現型のデータを関連付ける
mt2 = mt.annotate_cols(pheno=table_pheno[mt.s], line=table_line[mt.s])

In [6]:
# 表現型データがある系統の遺伝子型データを抜き出す
mt4 = mt2.semi_join_cols(table_pheno)
mt4.count()

2021-06-29 01:13:09 Hail: INFO: Coerced sorted dataset


(38769, 388)

In [7]:
mt4.ag

AttributeError: ignored

In [8]:
score = (mt4.GT.n_alt_alleles() - 1)

In [9]:
hl.export_vcf(mt4, 'mt4.vcf')

2021-06-29 01:13:21 Hail: WARN: export_vcf: ignored the following fields:
    'pheno' (column)
    'line' (column)
2021-06-29 01:13:23 Hail: INFO: Coerced sorted dataset
2021-06-29 01:13:36 Hail: INFO: merging 2 files totalling 59.0M...
2021-06-29 01:13:36 Hail: INFO: while writing:
    mt4.vcf
  merge time: 220.072ms


In [10]:
!pip install scikit-allel
import allel

     |████████████████████████████████| 5.7MB 7.0MB/s 


In [11]:
callset = allel.read_vcf('./RnData/HDRA-G6-4-RDP1-RDP2-NIAS.AGCT_MAF005MIS001NR.vcf')

In [12]:
callset.keys()

dict_keys(['samples', 'calldata/GT', 'variants/ALT', 'variants/CHROM', 'variants/FILTER_PASS', 'variants/ID', 'variants/POS', 'variants/QUAL', 'variants/REF'])

In [13]:
from sklearn.linear_model import Lasso
import numpy as np

In [14]:
fertile_florets_per_plant = mt4.pheno['Panicle.number.per.plant'] * mt4.pheno['Florets.per.panicle'] * mt4.pheno['Panicle.fertility']

In [15]:
ffpp = np.asarray(fertile_florets_per_plant.collect())

2021-06-29 01:15:00 Hail: INFO: Coerced sorted dataset


In [16]:
callset['calldata/GT'].shape, ffpp.shape

((38769, 1568, 2), (388,))

In [17]:
table_line.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    '': str 
    'NSFTV.ID': int32 
    'GSOR.ID': int32 
    'IRGC.ID': str 
    'Accession.Name': str 
    'Country.of.origin': str 
    'Latitude': float64 
    'Longitude': float64 
    'Sub.population': str 
----------------------------------------
Key: ['']
----------------------------------------


In [18]:
names = table_pheno[''].collect()

In [19]:
names = mt.s.collect()

2021-06-29 01:15:15 Hail: INFO: Coerced sorted dataset


In [ ]:
mt.s.show()

2020-06-24 06:43:47 Hail: INFO: Coerced sorted dataset


""
s
str
"""IRGC121316@c88dcbba.0"""
"""IRGC121578@950ba1f2.0"""
"""IRGC121491@0b7d031b.0"""
"""IRGC121440@d02dfe4e.0"""
"""IRGC121896@89688646.0"""
"""IRGC121323@57c71a3b.0"""
"""IRGC121251@392016d6.0"""
"""IRGC121902@f1be6ab1.0"""


In [ ]:
for name in names:
  if 'IRGC' in name:
    print(name)

IRGC121316@c88dcbba.0
IRGC121578@950ba1f2.0
IRGC121491@0b7d031b.0
IRGC121440@d02dfe4e.0
IRGC121896@89688646.0
IRGC121323@57c71a3b.0
IRGC121251@392016d6.0
IRGC121902@f1be6ab1.0
IRGC124483@9a9187e6.0
IRGC121314@6c2404e4.0
IRGC121899@60808a33.0
IRGC121864@0a12f8f9.0
IRGC121321@7495fecd.0
IRGC121773@de22ff14.0
IRGC121262@16ef3c90.0
IRGC121265@26863647.0
IRGC127012@2746fbbe.0
IRGC121472@30b72e37.0
IRGC121467@0b170025.0
IRGC121084@7f32098d.0
IRGC127014@7395f5f2.0
IRGC121944@c10dc5f6.0
IRGC121426@7f495a55.0
IRGC121352@608b0d34.0
IRGC121330@244f1a44.0
SONA_CHUR::IRGC53931-1@669c0aed.0
IRGC121332@780db140.0
IRGC127020@65e9acfc.0
IRGC117551@ea0b205a.0
IRGC121509@d8ee9654.0
IRGC121909@d00521f4.0
IRGC121455@6461d175.0
IRGC121499@46aa9c6a.0
IRGC117425@d00c1e13.0
IRGC121223@1ded3870.0
IRGC117426@61659fb4.0
IRGC121898@d4f656cf.0
IRGC121631@8366125c.0
IRGC117475@9517bcf3.0
IRGC117494@92044b56.0
IRGC121559@51a3d19a.0
IRGC121324@deaa5daa.0
IRGC121155@994ddbe2.0
IRGC121114@d9812864.0
IRGC121270@63b37a65.

In [20]:
callset['variants/CHROM'].shape, callset['variants/POS'].shape

((38769,), (38769,))

In [21]:
callset['variants/ID']

array(['SNP-1.8562.', 'SNP-1.18594.', 'SNP-1.24921.', ...,
       'SNP-12.27474843.', 'SNP-12.27476950.', 'SNP-12.27493334.'],
      dtype=object)

In [22]:
callset['calldata/GT'].shape, callset['samples'].shape

((38769, 1568, 2), (1568,))

In [ ]:
ids

['NSFTV100@1f10be3d.0',
 'NSFTV101@9f782e9e.0',
 'NSFTV102@6e26f4cc.0',
 'NSFTV103@8c76404c.0',
 'NSFTV104@8629f76c.0',
 'NSFTV105@16463092.0',
 'NSFTV106@8a06320f.0',
 'NSFTV107@7b7a0d82.0',
 'NSFTV108@e3b049a9.0',
 'NSFTV10@2e1c9c87.0',
 'NSFTV110@11bf5114.0',
 'NSFTV112@531e23fa.0',
 'NSFTV113@7a723d9e.0',
 'NSFTV114@eac19fb8.0',
 'NSFTV115@0bca95e0.0',
 'NSFTV116@6cedf6aa.0',
 'NSFTV117@5b144b9c.0',
 'NSFTV118@71bd9426.0',
 'NSFTV119@3aa51818.0',
 'NSFTV11@1d0066e2.0',
 'NSFTV120@8e6220e5.0',
 'NSFTV121@280279b3.0',
 'NSFTV122@b6dc1bcc.0',
 'NSFTV123@714ac141.0',
 'NSFTV124@6c91b63d.0',
 'NSFTV125@63f298ba.0',
 'NSFTV126@0f6a67da.0',
 'NSFTV128@76a1efc9.0',
 'NSFTV129@8fafd383.0',
 'NSFTV130@a796716d.0',
 'NSFTV131@d09d62e7.0',
 'NSFTV132@02cc7c6d.0',
 'NSFTV133@1a95985b.0',
 'NSFTV134@4ab486ec.0',
 'NSFTV136@d72ee9ba.0',
 'NSFTV137@8653bbdb.0',
 'NSFTV138@1a946dc6.0',
 'NSFTV139@bd0d322b.0',
 'NSFTV13@660f0236.0',
 'NSFTV140@85551f9c.0',
 'NSFTV141@a8319fc6.0',
 'NSFTV142@806c51cc

In [25]:
ids = table_line[''].collect()

In [26]:
indices, = np.where(np.isin(callset['samples'], ids))

In [27]:
callset2 = {k:v[indices] for k, v in callset.items()}

In [28]:
callset2['samples'].shape

(388,)

In [ ]:
callset2['calldata/GT'].shape

(388, 1568, 2)

In [ ]:
ffpp

array([8.002506243321962, 10.860646171031092, 10.520902076555744,
       11.693599378771145, 10.611359266287991, 10.909622235444594,
       10.965227275835813, 12.926598772302214, 12.782068342976464,
       11.890468959130157, 12.617006248709242, 15.005686531385699,
       13.328284193692571, 12.52930810271492, 11.508943430481112,
       13.768173436732404, 12.057367813651, 13.64107144728453,
       14.700437511934359, 14.137819625316544, 14.584768819118628,
       14.793305338149013, 9.767110849196527, 13.197388455028555,
       8.447640211825236, 10.985601014018274, 13.527178070798362,
       15.11476633748833, 12.031682638188881, 11.709327477763765,
       11.462489793575498, 15.18294457472342, 13.900583898142134,
       13.193044751832774, 10.938262497380355, 13.784867091059212,
       10.553596647654402, 13.463852035938531, 11.624831723740845,
       12.556630182316454, 12.839613488511763, 11.43090884243612, None,
       13.862739785257247, 10.690915870967677, 11.85747976925786,
 

In [29]:
mt5 = mt4.collect_cols_by_key()

In [30]:
mt5.cols().show()

2021-06-29 01:16:34 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2021-06-29 01:16:35 Hail: INFO: Coerced sorted dataset


,,
s,pheno,line
str,"array<struct{`Flowering.time.at.Arkansas`: float64, `Flowering.time.at.Faridpur`: int32, `Flowering.time.at.Aberdeen`: int32, `FT.ratio.of.Arkansas.Aberdeen`: float64, `FT.ratio.of.Faridpur.Aberdeen`: float64, `Culm.habit`: float64, `Leaf.pubescence`: int32, `Flag.leaf.length`: float64, `Flag.leaf.width`: float64, `Awn.presence`: int32, `Panicle.number.per.plant`: float64, `Plant.height`: float64, `Panicle.length`: float64, `Primary.panicle.branch.number`: float64, `Seed.number.per.panicle`: float64, `Florets.per.panicle`: float64, `Panicle.fertility`: float64, `Seed.length`: float64, `Seed.width`: float64, `Seed.volume`: float64, `Seed.surface.area`: float64, `Brown.rice.seed.length`: float64, `Brown.rice.seed.width`: float64, `Brown.rice.surface.area`: float64, `Brown.rice.volume`: float64, `Seed.length.width.ratio`: float64, `Brown.rice.length.width.ratio`: float64, `Seed.color`: int32, `Pericarp.color`: int32, `Straighthead.suseptability`: float64, `Blast.resistance`: int32, `Amylose.content`: float64, `Alkali.spreading.value`: float64, `Protein.content`: float64, `Year07Flowering.time.at.Arkansas`: float64, `Year06Flowering.time.at.Arkansas`: float64}>","array<struct{`NSFTV.ID`: int32, `GSOR.ID`: int32, `IRGC.ID`: str, `Accession.Name`: str, `Country.of.origin`: str, Latitude: float64, Longitude: float64, `Sub.population`: str}>"
"""NSFTV100@1f10be3d.0""","[(8.41e+01,78,122,6.89e-01,6.39e-01,1.67e+00,0,2.76e+01,1.61e+00,0,2.98e+00,1.15e+02,2.39e+01,1.16e+01,5.03e+00,5.34e+00,7.35e-01,8.14e+00,3.38e+00,2.44e+00,3.85e+00,5.94e+00,2.89e+00,3.47e+00,6.90e+00,2.41e+00,2.06e+00,0,0,6.50e+00,3,1.06e+01,6.96e+00,8.70e+00,8.15e+01,8.93e+01)]","[(100,301092,""117797"",""Lacrosse"",""United States"",3.47e+01,-9.23e+01,""ADMIX"")]"
"""NSFTV101@9f782e9e.0""","[(8.62e+01,79,88,9.79e-01,8.98e-01,3.00e+00,0,2.76e+01,1.45e+00,0,2.82e+00,8.62e+01,2.75e+01,1.11e+01,4.89e+00,5.21e+00,7.30e-01,9.63e+00,2.64e+00,2.12e+00,3.78e+00,7.15e+00,2.25e+00,3.39e+00,4.92e+00,3.64e+00,3.17e+00,0,0,3.33e+00,8,2.05e+01,6.00e+00,9.50e+00,8.00e+01,9.23e+01)]","[(101,301093,""117802"",""Lemont"",""United States"",3.47e+01,-9.23e+01,""TRJ"")]"
"""NSFTV102@6e26f4cc.0""","[(NA,NA,165,NA,NA,3.00e+00,NA,2.79e+01,1.10e+00,1,3.48e+00,1.06e+02,3.08e+01,1.05e+01,4.60e+00,4.87e+00,7.65e-01,9.81e+00,2.47e+00,2.03e+00,3.75e+00,7.07e+00,2.04e+00,3.30e+00,4.13e+00,3.97e+00,3.46e+00,0,0,6.84e+00,1,2.13e+01,5.00e+00,8.70e+00,NA,NA)]","[(102,301094,""117619"",""Leung Pratew"",""Thailand"",1.59e+01,1.01e+02,""IND"")]"
"""NSFTV103@8c76404c.0""","[(8.40e+01,78,108,7.78e-01,7.22e-01,2.50e+00,1,3.02e+01,1.05e+00,0,2.96e+00,9.51e+01,2.41e+01,9.78e+00,4.88e+00,5.00e+00,8.89e-01,7.53e+00,3.53e+00,2.40e+00,3.78e+00,5.30e+00,3.05e+00,3.39e+00,6.67e+00,2.13e+00,1.74e+00,0,0,5.34e+00,4,1.70e+01,6.92e+00,7.75e+00,8.40e+01,8.40e+01)]","[(103,301095,""117807"",""Luk Takhar"",""Afghanistan"",3.39e+01,6.77e+01,""TEJ"")]"
"""NSFTV104@8629f76c.0""","[(7.75e+01,70,74,1.05e+00,9.46e-01,5.50e+00,NA,2.76e+01,1.13e+00,1,3.14e+00,1.10e+02,2.36e+01,1.14e+01,4.89e+00,5.03e+00,8.73e-01,6.61e+00,3.32e+00,2.12e+00,3.56e+00,4.49e+00,2.90e+00,3.17e+00,5.12e+00,1.99e+00,1.55e+00,0,0,7.33e+00,7,8.59e+00,6.88e+00,8.45e+00,7.20e+01,8.30e+01)]","[(104,301096,""117811"",""Mansaku"",""Japan"",3.62e+01,1.38e+02,""TEJ"")]"
"""NSFTV105@16463092.0""","[(8.05e+01,80,81,9.94e-01,9.88e-01,6.33e+00,1,3.18e+01,1.53e+00,0,3.73e+00,1.31e+02,2.50e+01,8.00e+00,5.04e+00,5.12e+00,9.29e-01,7.61e+00,2.79e+00,1.97e+00,3.58e+00,5.70e+00,2.40e+00,3.23e+00,4.47e+00,2.73e+00,2.38e+00,0,0,8.09e+00,4,2.46e+01,4.83e+00,7.20e+00,7.90e+01,8.20e+01)]","[(105,301097,""117620"",""Mehr"",""Iran"",3.24e+01,5.37e+01,""AUS"")]"
"""NSFTV106@8a06320f.0""","[(1.03e+02,75,108,9.52e-01,6.94e-01,3.00e+00,NA,3.02e+01,1.44e+00,0,3.49e+00,9.38e+01,2.59e+01,9.17e+00,4.89e+00,5.08e+00,8.26e-01,9.42e+00,2.93e+00,2.33e+00,3.88e+00,6.95e+00,2.49e+00,3.48e+00,5.97e+00,3.21e+00,2.79e+00,0,0,6.83e+00,0,1.41e+01,3.39e+00,7.45e+00,1.00e+02,1.06e

In [31]:
(mt4.GT.n_alt_alleles() - 1).show()

2021-06-29 01:16:42 Hail: INFO: Coerced sorted dataset
2021-06-29 01:16:43 Hail: INFO: Coerced sorted dataset
2021-06-29 01:16:45 Hail: INFO: Coerced sorted dataset


,,,,
,,'NSFTV80@02d095ba.0','NSFTV87@64fa0112.0','NSFTV96@32a6808e.0'
locus,alleles,,,
locus<GRCh37>,array<str>,int32,int32,int32
1:9563,"[""A"",""T""]",-1,-1,-1
1:19595,"[""G"",""A""]",-1,-1,-1
1:25922,"[""C"",""T""]",-1,-1,-1
1:26254,"[""A"",""T""]",-1,-1,-1
1:30214,"[""T"",""A""]",-1,-1,-1
1:31478,"[""C"",""T""]",-1,1,-1
1:32733,"[""T"",""G""]",-1,-1,-1


In [32]:
fertile_florets_per_plant = mt4.pheno['Panicle.number.per.plant'] * mt4.pheno['Florets.per.panicle'] * mt4.pheno['Panicle.fertility']

In [34]:
eigenvalues, pcs, loadings  = hl.hwe_normalized_pca(mt4.GT, k=6)

2021-06-29 01:17:18 Hail: INFO: Coerced sorted dataset
2021-06-29 01:17:25 Hail: INFO: hwe_normalized_pca: running PCA using 38768 variants.
2021-06-29 01:17:27 Hail: INFO: Coerced sorted dataset
2021-06-29 01:17:34 Hail: INFO: pca: running PCA with 6 components...


In [35]:
mt3 = mt4.annotate_cols(scores = pcs[mt4.s].scores)
p12 = hl.plot.scatter(mt3.scores[0],
                    mt3.scores[1],
                    label=mt3.line['Sub.population'],
                    title='PCA PC1-PC2', xlabel='PC1', ylabel='PC2')
p34 = hl.plot.scatter(mt3.scores[2],
                    mt3.scores[3],
                    label=mt3.line['Sub.population'],
                    title='PCA PC3-PC4', xlabel='PC3', ylabel='PC4')
p56 = hl.plot.scatter(mt3.scores[4],
                      mt3.scores[5],
                      label=mt3.line['Sub.population'],
                      title='PCA PC5-PC6', xlabel='PC5', ylabel='PC6')
show(p12)
show(p34)
show(p56)

2021-06-29 01:17:58 Hail: INFO: Coerced sorted dataset
2021-06-29 01:18:01 Hail: INFO: Coerced sorted dataset
2021-06-29 01:18:03 Hail: INFO: Coerced sorted dataset


In [36]:
mt3.line.show()

2021-06-29 01:18:11 Hail: INFO: Coerced sorted dataset


+-----------------------+---------------+--------------+------------------+
| s                     | line.NSFTV.ID | line.GSOR.ID | line.IRGC.ID     |
+-----------------------+---------------+--------------+------------------+
| str                   |         int32 |        int32 | str              |
+-----------------------+---------------+--------------+------------------+
| "NSFTV80@02d095ba.0"  |            80 |       301073 | "To be assigned" |
| "NSFTV87@64fa0112.0"  |            87 |       301079 | "117618"         |
| "NSFTV96@32a6808e.0"  |            96 |       301088 | "117792"         |
| "NSFTV100@1f10be3d.0" |           100 |       301092 | "117797"         |
| "NSFTV114@eac19fb8.0" |           114 |       301106 | "117823"         |
| "NSFTV140@85551f9c.0" |           140 |       301131 | "117879"         |
| "NSFTV206@37249a74.0" |           206 |       301197 | "117865"         |
| "NSFTV211@5497b233.0" |           211 |       301202 | "117917"         |
| "NSFTV217@915c0033.0" |           217 |       301208 | "117941"         |
| "NSFTV218@85da3a70.0" |           218 |       301209 | "117851"         |
+-----------------------+---------------+--------------+------------------+

+---------------------+------------------------+---------------+
| line.Accession.Name | line.Country.of.origin | line.Latitude |
+---------------------+------------------------+---------------+
| str                 | str                    |       float64 |
+---------------------+------------------------+---------------+
| "K 65"              | "Suriname"             |      3.92e+00 |
| "Keriting Tingii"   | "Indonesia"            |     -7.89e-01 |
| "KU115"             | "Thailand"             |      1.59e+01 |
| "Lacrosse"          | "United States"        |      3.47e+01 |
| "Nova"              | "United States"        |      3.47e+01 |
| "Saturn"            | "United States"        |      3.47e+01 |
| "Rojofotsy 738"     | "Madagascar"           |     -1.88e+01 |
| "Tokyo Shino Mochi" | "Japan"                |      3.62e+01 |
| "YRL-1"             | "Australia"            |     -2.53e+01 |
| "PI 298967-1"       | "Australia"            |     -2.53e+01 |
+---------------------+------------------------+---------------+

+----------------+---------------------+
| line.Longitude | line.Sub.population |
+----------------+---------------------+
|        float64 | str                 |
+----------------+---------------------+
|      -5.60e+01 | "ADMIX"             |
|       1.14e+02 | "ADMIX"             |
|       1.01e+02 | "ADMIX"             |
|      -9.23e+01 | "ADMIX"             |
|      -9.23e+01 | "ADMIX"             |
|      -9.23e+01 | "ADMIX"             |
|       4.69e+01 | "ADMIX"             |
|       1.38e+02 | "ADMIX"             |
|       1.34e+02 | "ADMIX"             |
|       1.34e+02 | "ADMIX"             |
+----------------+---------------------+
showing top 10 rows

In [37]:
# NA以外のデータを抽出し、RRM行列を計算する
mtt3 = mt3.filter_cols(mt3.pheno['Seed.length.width.ratio'] >= 0)
rrm = hl.realized_relationship_matrix(mtt3.GT)

2021-06-29 01:18:20 Hail: INFO: Coerced sorted dataset
2021-06-29 01:18:53 Hail: INFO: Wrote all 10 blocks of 38768 x 361 matrix with block size 4096.


In [38]:
mtt3 = mtt3.annotate_cols(scores = pcs[mtt3.s].scores)

In [39]:
model, p = hl.linear_mixed_model(
    y=mtt3.pheno['Seed.length.width.ratio'],
    x=[1.0, mtt3.scores[0], mtt3.scores[1], mtt3.scores[2], mtt3.scores[3]],
    k=rrm.to_numpy(),
    overwrite=True,
    p_path='p2.bm')

2021-06-29 01:19:09 Hail: INFO: Coerced sorted dataset
2021-06-29 01:19:10 Hail: INFO: wrote matrix with 361 rows and 361 columns as 1 block of size 4096 to p2.bm


In [40]:
model.fit()
model.h_sq

0.9040960512880675

In [41]:
rt = hl.linear_mixed_regression_rows(mtt3.GT.n_alt_alleles(), model)

2021-06-29 01:19:17 Hail: INFO: Coerced sorted dataset
2021-06-29 01:19:18 Hail: INFO: Coerced sorted dataset
2021-06-29 01:19:20 Hail: INFO: Coerced sorted dataset
2021-06-29 01:19:52 Hail: INFO: Wrote all 10 blocks of 38769 x 361 matrix with block size 4096.
2021-06-29 01:20:06 Hail: INFO: wrote matrix with 38769 rows and 361 columns as 10 blocks of size 4096 to /tmp/0A5gsPecxBMTVkPGwPtPDi


In [42]:
p = hl.plot.qq(rt.p_value)
show(p)

2021-06-29 01:20:50 Hail: INFO: Coerced sorted dataset
2021-06-29 01:20:53 Hail: INFO: Coerced sorted dataset
2021-06-29 01:20:56 Hail: INFO: Ordering unsorted dataset with network shuffle


In [43]:

len(rt.p_value.collect())

2021-06-29 01:21:04 Hail: INFO: Coerced sorted dataset
2021-06-29 01:21:06 Hail: INFO: Coerced sorted dataset


38769

In [44]:
p = hl.plot.manhattan(rt.p_value)
show(p)

2021-06-29 01:21:11 Hail: INFO: Coerced sorted dataset
2021-06-29 01:21:13 Hail: INFO: Coerced sorted dataset


In [45]:
hlmt.GT.n_alt_alleles() - 1

NameError: ignored

2020-06-04 14:45:14 Hail: INFO: Coerced sorted dataset
